<a href="https://colab.research.google.com/github/amburu4159/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.1.2'
os.environ['SPARK_VERSION']=spark_version
# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.2.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.2.tgz
!pip install -q findspark
# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3.2"
# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/universe amd64 Packages [1,424 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restricted amd64

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Personal_Care_Appliances_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   32114233|R1QX6706ZWJ1P5|B00OYRW4UE|     223980852|Elite Sportz Exer...|Personal_Care_App...|          5|            0|          0|   N|                Y|Good quality. Shi...|Exactly as descri...| 2015-08-31|
|         US|   18125776|R3QWMLJHIW6P37|B0000537JQ|     819771537|     Ezy Dose Weekly|Personal_Care_App

In [6]:
#filter DF for total_votes >= 20
df = df.filter(df.total_votes >= 20)
df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26034089|R2PT2X8FMYFCV3|B00PV2QSLW|     991555077|Pride Mobility Go...|Personal_Care_App...|          5|           39|         41|   N|                Y|          Five Stars|Nice not too heav...| 2015-08-18|
|         US|   19214268| RGCUQSH80SYY7|B00H8ZTLLY|     548146182|doTERRA On Guard ...|Personal_Care_App

In [7]:
#filter DF for helpful_votes ratio>= 50%
df_helpful = df.filter((df.helpful_votes / df.total_votes) >= 0.5)
df_helpful.show(10)



+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26034089|R2PT2X8FMYFCV3|B00PV2QSLW|     991555077|Pride Mobility Go...|Personal_Care_App...|          5|           39|         41|   N|                Y|          Five Stars|Nice not too heav...| 2015-08-18|
|         US|   19214268| RGCUQSH80SYY7|B00H8ZTLLY|     548146182|doTERRA On Guard ...|Personal_Care_App

In [8]:
#filter DF created in Step 2 and create new df that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
paid_df = df_helpful.filter(df_helpful.vine == 'Y')
paid_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   35353346|R3R2RHQWEKX5NL|B00DSICT1Y|     427004417|uComfy Shiatsu Fo...|Personal_Care_App...|          5|           81|         89|   Y|                N|You can hear your...|First off one bit...| 2014-01-13|
|         US|   50929595|R1RZ4JZO8DAP7H|B00DSICT1Y|     427004417|uComfy Shiatsu Fo...|Personal_Care_App

In [9]:
#filter DF created in Step 2 and create new df that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'N'.
unpaid_df = df_helpful.filter(df_helpful.vine == 'N')
unpaid_df.show(10)


+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   26034089|R2PT2X8FMYFCV3|B00PV2QSLW|     991555077|Pride Mobility Go...|Personal_Care_App...|          5|           39|         41|   N|                Y|          Five Stars|Nice not too heav...| 2015-08-18|
|         US|   19214268| RGCUQSH80SYY7|B00H8ZTLLY|     548146182|doTERRA On Guard ...|Personal_Care_App

In [11]:
#determine the total number of reviews -PAID
total_reviews_paid = paid_df.count()
total_reviews_paid

3

In [12]:
#determine the number of 5-star reviews - PAID
five_star_reviews_paid = paid_df.filter(paid_df.star_rating == 5).count()
five_star_reviews_paid

2

In [14]:
#determine the percentage of 5-star reviews - PAID
five_star_percent_paid = (five_star_reviews_paid / total_reviews_paid) * 100
five_star_percent_paid

66.66666666666666

In [15]:
#determine the total number of reviews -UNPAID
total_reviews_unpaid = unpaid_df.count()
total_reviews_unpaid

3094

In [16]:
#determine the number of 5-star reviews - UNPAID
five_star_reviews_unpaid = unpaid_df.filter(unpaid_df.star_rating == 5).count()
five_star_reviews_unpaid

1704

In [18]:
#determine the percentage of 5-star reviews - UNPAID
five_star_percent_unpaid = (five_star_reviews_unpaid / total_reviews_unpaid) * 100
five_star_percent_unpaid

55.07433742727861